In [64]:
from time import perf_counter, sleep
from functools import wraps
import random

In [1]:
class Person:
    def __call__(self):
        print('__call__ called...')

In [2]:
p = Person()
p()

__call__ called...


In [3]:
class Person1:
    pass
p1 = Person1()
p1()

TypeError: 'Person1' object is not callable

In [4]:
type(p), type(p1)

(__main__.Person, __main__.Person1)

### Partial from functools

In [5]:
from functools import partial

In [7]:
type(partial), type(Person)

(type, type)

this means that partial is a callable class, not a function

In [9]:
def my_func(a,b,c):
    return a,b,c
type(my_func)

function

In [12]:
partial_func = partial(my_func, 10, 20)
type(partial_func)

functools.partial

In [13]:
partial_func(30)

(10, 20, 30)

### Recreate Partial

In [15]:
class Partial:
    def __init__(self, func, *args):
        self._func = func
        self._args = args
    def __call__(self, *args):
        return self._func(*self._args, *args)

In [16]:
partial_func = Partial(my_func, 10, 20)

In [17]:
type(partial_func)

__main__.Partial

In [18]:
partial_func(30)

(10, 20, 30)

### Check if callable

In [19]:
callable(print)

True

In [20]:
callable(int), type(int)

(True, type)

In [21]:
type(partial_func), callable(partial_func)

(__main__.Partial, True)

In [22]:
callable(Partial), callable(Person)

(True, True)

In [24]:
class Person:
    pass
p = Person()
callable(p)

False

### Implementing caching mechanism

dictionary, that will be used for caching

In [25]:
from collections import defaultdict

In [26]:
def default_value():
    return "N/A"

In [27]:
d = defaultdict(default_value)

In [28]:
d['a']

'N/A'

In [29]:
d['b']=100

In [30]:
d["b"]

100

In [31]:
d.items()

dict_items([('a', 'N/A'), ('b', 100)])

## how many times nonexisting key was requested

#### a try to use global counter

In [39]:
miss_counter = 0

In [40]:
def default_value():
    global miss_counter
    miss_counter += 1
    return "N/A"

In [41]:
d = defaultdict(default_value)
d['a']

'N/A'

In [42]:
d['b']=100
d["b"]

100

In [43]:
miss_counter

1

### try a class-based approach

In [44]:
class DefaultValue:
    def __init__(self):
        self.counter = 0
    def __iadd__(self, other):
        if isinstance(other, int):
            self.counter += other
            return self
        raise ValueError('Can only increment with an integer value.')
            
    

In [45]:
default_value1 = DefaultValue()
default_value1.counter

0

In [46]:
default_value1 += 1
default_value1.counter

1

#### we need a callable for the defaultdict

In [47]:
class DefaultValue:
    def __init__(self):
        self.counter = 0
#     def __iadd__(self, other):
#         if isinstance(other, int):
#             self.counter += other
#             return self
#         raise ValueError('Can only increment with an integer value.')
    def __call__(self):
        self.counter += 1
        return 'N/A'

In [48]:
def_1 = DefaultValue()
def_2 = DefaultValue()

In [49]:
cache1 = defaultdict(def_1)
cache2 = defaultdict(def_2)

In [50]:
cache1['a'], cache1['b']

('N/A', 'N/A')

In [51]:
def_1.counter

2

In [52]:
cache2['a']

'N/A'

In [53]:
def_2.counter

1

#### little bit more flexible

In [54]:
class DefaultValue:
    def __init__(self, default_value):
        self.default_value = default_value
        self.counter = 0

    def __call__(self):
        self.counter += 1
        return self.default_value

In [55]:
def_1 = DefaultValue(None)
def_2 = DefaultValue(0)

In [56]:
cache1 = defaultdict(def_1)
cache2 = defaultdict(def_2)

In [57]:
cache1['a'], cache1['b']

(None, None)

In [58]:
def_1.counter

2

In [59]:
cache2['a']

0

In [60]:
def_2.counter

1

# create decorator 

### how many times a function was called and how long it runs on average

#### a closure-based decorator

In [66]:
def profiler(fn):
    counter = 0
    total_elapsed = 0
    avg_time = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal counter
        nonlocal total_elapsed
        nonlocal avg_time
        counter += 1
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        total_elapsed += (end - start)
        avg_time = total_elapsed / counter
        return result
    inner.counter = counter #won't work
    inner.avg_time = avg_time #won't work
    
    #we made inner.counter the VALUE of counter at the time
    #the decorator function was called - 
    #this is not the counter value that we keep updating
    
    return inner

In [ ]:
random.seed(0)

In [68]:
@profiler
def func1():
    sleep(random.random())
    

In [69]:
 func1(),  func1()

(None, None)

In [70]:
 func1.counter # doesn't work

0

#### let's fix it

In [71]:
def profiler(fn):
    _counter = 0
    _total_elapsed = 0
    _avg_time = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal _counter
        nonlocal _total_elapsed
        nonlocal _avg_time
        _counter += 1
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        _total_elapsed += (end - start)
        _avg_time = _total_elapsed / _counter
        return result
    
    def counter():
        # this will now be a closure with a cell pointing to the object
        # _counter points to.
        return _counter
    def avg_time():
        return _avg_time
        
    
    inner.counter = counter #won't work
    inner.avg_time = avg_time #won't work
    return inner

In [72]:
@profiler
def func1():
    sleep(random.random())
func1(),  func1()    

(None, None)

In [77]:
func1.counter()

2

In [74]:
func1.avg_time()

0.6095126500040351

#### class-based decorator

In [78]:
class Profiler:
    def __init__(self, fn):
        self.counter = 0
        self.total_elapsed = 0
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        self.counter += 1
        start = perf_counter()
        result = self.fn(*args, **kwargs)
        end = perf_counter()
        self.total_elapsed += (end-start)
        return result
    
    @property
    def avg_time(self):
        return self.total_elapsed / self.counter 
        

In [79]:
@Profiler
def func1(a,b):
    sleep(random.random())
    return a,b

In [82]:
func1(2,3),  func1(0,0)    

((2, 3), (0, 0))

In [84]:
func1.counter, func1.avg_time

(2, 0.6227376499991806)